In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np

### Read recipe inputs

Matches

In [0]:
matches_ds = dataiku.Dataset("matches")
matches = matches_ds.get_dataframe(infer_with_pandas=False, bool_as_str=True)
matches

Corrected data

In [0]:
corrected_data = dataiku.Dataset("matches_uncertain_editlog_pivoted").get_dataframe()
corrected_data

### Apply edits to matches with distance > 0 or undefined

First let's have a look at uncertain matches, before edits

In [0]:
mask_uncertain = (matches["distance_initial_match"]>0) | (matches["distance_initial_match"].isnull())
matches_uncertain_unedited = matches[mask_uncertain]
matches_uncertain_unedited

And after edits

In [0]:
commons = dataiku.import_from_plugin("editable-via-webapp", "commons")
primary_keys = ["id_companies_ref"]
corrected_data["last_edit_date"] = ""
matches_uncertain_edited = commons.merge_edits_from_all_df(matches_uncertain_unedited, corrected_data, primary_keys)
matches_uncertain_edited

### Define the new "matches uncertain"

Rows that are still not marked as reviewed

In [0]:
matches_uncertain = matches_uncertain_edited[matches_uncertain_edited["reviewed"]!=True]
matches_uncertain

### Define "matches certain"

This is made of two parts:

* Matches that were previously uncertain but that are now marked as reviewed

In [0]:
matches_uncertain_reviewed = matches_uncertain_edited[matches_uncertain_edited["reviewed"]==True] # those where reviewed is True
matches_uncertain_reviewed

* Exact matches

In [0]:
mask_exact_match = matches["distance_initial_match"]==0
matches[mask_exact_match]

We concatenate both

In [0]:
matches_certain = pd.concat([matches_uncertain_reviewed, matches[mask_exact_match]])

### Write recipe outputs

In [0]:
matches_uncertain_ds = dataiku.Dataset("matches_uncertain")
matches_uncertain_ds.write_schema(matches_ds.read_schema(), dropAndCreate=True)
matches_uncertain_ds.write_dataframe(matches_uncertain, infer_schema=False)

matches_certain_ds = dataiku.Dataset("matches_certain")
matches_certain_ds.write_schema(matches_ds.read_schema(), dropAndCreate=True)
matches_certain_ds.write_dataframe(matches_certain, infer_schema=False)